In [78]:
git init

SyntaxError: invalid syntax (2830201818.py, line 1)

In [56]:
##移動angle、bar 檔案 全體版
import os
import shutil
import pandas as pd

def copy_folders_from_excel(excel_path, source_root, dest_root):
    df = pd.read_excel(excel_path, header=None)
    categories = [f"Category_{i+1}" for i in range(5)]

    for idx, row in df.iterrows():
        first_level = str(row[0])
        second_level = str(row[1])
        source_folder = os.path.join(source_root, first_level, second_level)

        if not os.path.exists(source_folder):
            print(f"❌ Source folder not found: {source_folder}")
            continue

        for i, category in enumerate(categories, start=2):
            if row[i] == 1:
                dest_folder = os.path.join(dest_root, category, second_level)

                # ✅ 新增判斷：若目標資料夾不存在，就整筆跳過
                if not os.path.exists(dest_folder):
                    print(f"⚠️ Destination folder not found, skipped: {dest_folder}")
                    continue

                for subfolder in ["angle_vision1"]:
                    src_subfolder = os.path.join(source_folder, subfolder)
                    dest_subfolder = os.path.join(dest_folder, subfolder)

                    if os.path.exists(src_subfolder):
                        shutil.copytree(src_subfolder, dest_subfolder, dirs_exist_ok=True)
                        print(f"✅ Copied {src_subfolder} → {dest_subfolder}")
                    else:
                        print(f"⚠️ Subfolder not found, skipped: {src_subfolder}")

# 使用方式
excel_path = "D:\\labdata\\MOCAP\\recordings_copy\\folder_structure.xlsx"
source_root = "D:\\labdata\\MOCAP\\recordings_copy"
dest_root = "D:\\labdata\\MOCAP\\deadlift_system\\datasets"

copy_folders_from_excel(excel_path, source_root, dest_root)



❌ Source folder not found: D:\labdata\MOCAP\recordings_copy\Folder\Subfolder
✅ Copied D:\labdata\MOCAP\recordings_copy\受試者10\recording_20241204_100047\angle_vision1 → D:\labdata\MOCAP\deadlift_system\datasets\Category_1\recording_20241204_100047\angle_vision1
✅ Copied D:\labdata\MOCAP\recordings_copy\受試者10\recording_20241204_100314\angle_vision1 → D:\labdata\MOCAP\deadlift_system\datasets\Category_2\recording_20241204_100314\angle_vision1
✅ Copied D:\labdata\MOCAP\recordings_copy\受試者10\recording_20241204_100527\angle_vision1 → D:\labdata\MOCAP\deadlift_system\datasets\Category_3\recording_20241204_100527\angle_vision1
✅ Copied D:\labdata\MOCAP\recordings_copy\受試者10\recording_20241204_101023\angle_vision1 → D:\labdata\MOCAP\deadlift_system\datasets\Category_4\recording_20241204_101023\angle_vision1
✅ Copied D:\labdata\MOCAP\recordings_copy\受試者10\recording_20241204_101150\angle_vision1 → D:\labdata\MOCAP\deadlift_system\datasets\Category_5\recording_20241204_101150\angle_vision1
✅ Copied

In [27]:
##合併並插值檔案 單一版
import os
import numpy as np
import pandas as pd

def linear_interpolate_fixed_length(df, frame_col, target_length=110):
    df.set_index(frame_col, inplace=True)
    new_index = np.linspace(df.index.min(), df.index.max(), target_length)
    df = df.reindex(df.index.union(new_index)).interpolate(method='linear').loc[new_index]
    df.reset_index(inplace=True, drop=True)
    return df

def merge_and_interpolate(angle_path, bar_path, output_path, target_length=110):
    angle_files = sorted([f for f in os.listdir(angle_path) if f.startswith("angle_skeleton_")])
    bar_files = sorted([f for f in os.listdir(bar_path) if f.startswith("bar_")])
    
    for angle_file, bar_file in zip(angle_files, bar_files):
        angle_df = pd.read_csv(os.path.join(angle_path, angle_file), header=None)
        bar_df = pd.read_csv(os.path.join(bar_path, bar_file), header=None)
        
        angle_df = angle_df.iloc[:, [1, 2, 3, 4, 5]]
        bar_df = bar_df.iloc[:, [1, 2]]
        
        merged_df = pd.concat([angle_df, bar_df], axis=1)
        merged_df = linear_interpolate_fixed_length(merged_df, merged_df.index, target_length)
        
        output_file = os.path.join(output_path, f"merged_{angle_file.replace('angle_skeleton_', '')}")
        merged_df.to_csv(output_file, index=False, header=False, float_format='%.2f')
    
    print("Processing complete. Merged files saved in", output_path)


# Example usage:
path = 'D:\\labdata\\MOCAP\\train_data\\Category_1\\recording_20241121_101357'
out = path + '\\out'
merge_and_interpolate(path + '\\angle',path + '\\bar', out, target_length=110)


Processing complete. Merged files saved in D:\labdata\MOCAP\train_data\Category_1\recording_20241121_101357\out


In [29]:
##合併並插值檔案 全體版
import os
import numpy as np
import pandas as pd

def linear_interpolate_fixed_length(df, frame_col=0, target_length=110):
    """對 DataFrame 進行線性內插，使其長度固定為 target_length。"""
    df.set_index(df.columns[frame_col], inplace=True)  # 設定時間軸為索引
    new_index = np.linspace(df.index.min(), df.index.max(), target_length)
    df = df.reindex(df.index.union(new_index)).interpolate(method='linear').loc[new_index]
    df.reset_index(drop=True, inplace=True)
    return df

def merge_and_interpolate(angle_path, bar_path, output_path, target_length=110):
    """合併 angle 和 bar 資料夾內的檔案，並進行線性內插。"""
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    angle_files = sorted([f for f in os.listdir(angle_path) if f.startswith("angle_")])
    bar_files = sorted([f for f in os.listdir(bar_path) if f.startswith("bar_")])
    
    # 確保檔案名稱匹配
    angle_dict = {f.split("_")[-1]: f for f in angle_files}
    bar_dict = {f.split("_")[-1]: f for f in bar_files}
    
    common_keys = set(angle_dict.keys()) & set(bar_dict.keys())  # 找到對應的檔案

    for key in sorted(common_keys):
        angle_file = angle_dict[key]
        bar_file = bar_dict[key]

        try:
            angle_df = pd.read_csv(os.path.join(angle_path, angle_file), header=None)
            bar_df = pd.read_csv(os.path.join(bar_path, bar_file), header=None)
            
            # 選擇需要的欄位
            angle_df = angle_df.iloc[:, [0, 1, 2, 3]  # 包含 frame_col (假設是第 0 欄)
            bar_df = bar_df.iloc[:, [1, 2]]

            merged_df = pd.concat([angle_df, bar_df], axis=1)

            # 進行內插，確保 frame_col 為索引
            merged_df = linear_interpolate_fixed_length(merged_df, frame_col=0, target_length=target_length)

            output_file = os.path.join(output_path, f"merged_{key}")
            merged_df.to_csv(output_file, index=False, header=False, float_format="%.4f")

            print(f"Processed: {angle_file} + {bar_file} -> {output_file}")

        except Exception as e:
            print(f"Error processing {angle_file} and {bar_file}: {e}")

def process_all_folders(root_path, target_length=110):
    """遍歷 root_path 內的所有子資料夾，並處理其中的 angle 和 bar。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            angle_path = os.path.join(recording_path, "angle_int")
            bar_path = os.path.join(recording_path, "bar")
            output_path = os.path.join(recording_path, "out")

            if os.path.exists(angle_path) and os.path.exists(bar_path):
                print(f"Processing {recording_path} ...")
                merge_and_interpolate(angle_path, bar_path, output_path, target_length)

# Example usage
root_path = 'D:\\labdata\\MOCAP\\1234'
# process_all_folders(root_path, target_length=110)


In [61]:
import os
import numpy as np
import pandas as pd

def linear_interpolate_fixed_length(df, frame_col=0, target_length=110):
    """對 DataFrame 進行線性內插，使其長度固定為 target_length。"""
    df.set_index(df.columns[frame_col], inplace=True)  # 設定時間軸為索引
    new_index = np.linspace(df.index.min(), df.index.max(), target_length)
    df = df.reindex(df.index.union(new_index)).interpolate(method='linear').loc[new_index]
    df.reset_index(drop=True, inplace=True)
    return df

def merge_and_interpolate(angle_path, bar_path, vision_path, output_path, target_length=110):
    """合併 angle、bar 和 angle_vision1 的檔案，並進行線性內插。"""
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    angle_files = sorted([f for f in os.listdir(angle_path) if f.startswith("angle_skeleton_")])
    bar_files = sorted([f for f in os.listdir(bar_path) if f.startswith("bar_")])
    vision_files = sorted([f for f in os.listdir(vision_path) if f.startswith("angle_skeleton_")])

    # 建立檔案名稱對應字典（使用最後一段名稱作為 key）
    angle_dict = {f.split("_")[-1]: f for f in angle_files}
    bar_dict = {f.split("_")[-1]: f for f in bar_files}
    vision_dict = {f.split("_")[-1]: f for f in vision_files}

    # 找出三者都有的檔案
    common_keys = set(angle_dict.keys()) & set(bar_dict.keys()) & set(vision_dict.keys())

    for key in sorted(common_keys):
        angle_file = angle_dict[key]
        bar_file = bar_dict[key]
        vision_file = vision_dict[key]

        try:
            angle_df = pd.read_csv(os.path.join(angle_path, angle_file), header=None)
            bar_df = pd.read_csv(os.path.join(bar_path, bar_file), header=None)
            vision_df = pd.read_csv(os.path.join(vision_path, vision_file), header=None)

            # 選擇需要的欄位
            angle_df = angle_df.iloc[:, [0, 1, 2, 3]]  # 含 frame
            bar_df = bar_df.iloc[:, [1, 2]]
            vision_df = vision_df.iloc[:, [0, 1]]  # frame + 欲加欄位

            # 把 vision 欄位加到 angle 後，因為 frame 對齊，故只取 vision 的第 1 欄
            merged_df = pd.concat([angle_df, bar_df, vision_df.iloc[:, 1]], axis=1)

            # 進行內插
            merged_df = linear_interpolate_fixed_length(merged_df, frame_col=0, target_length=target_length)

            # 輸出
            output_file = os.path.join(output_path, f"merged_{key}")
            merged_df.to_csv(output_file, index=False, header=False, float_format="%.4f")

            print(f"Processed: {angle_file}, {bar_file}, {vision_file} -> {output_file}")

        except Exception as e:
            print(f"Error processing {angle_file}, {bar_file}, {vision_file}: {e}")

def process_all_folders(root_path, target_length=110):
    """遍歷 root_path 內的所有子資料夾，並處理 angle、bar 和 angle_vision1 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue

            angle_path = os.path.join(recording_path, "angle")
            bar_path = os.path.join(recording_path, "bar")
            vision_path = os.path.join(recording_path, "angle_vision1")
            output_path = os.path.join(recording_path, "out")

            if os.path.exists(angle_path) and os.path.exists(bar_path) and os.path.exists(vision_path):
                print(f"Processing {recording_path} ...")
                merge_and_interpolate(angle_path, bar_path, vision_path, output_path, target_length)

# Example usage
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'
# process_all_folders(root_path, target_length=110)


In [63]:
##去除極值
import os
import pandas as pd

def remove_outliers(df):
    """移除超過3個標準差的極端值，並用該列均值取代。"""
    mean = df.mean()
    std = df.std()
    
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    # 過濾掉極端值，超過範圍的值用均值取代
    df_filtered = df.mask((df < lower_bound) | (df > upper_bound), mean, axis=1)
    return df_filtered

def process_filtered_files(root_path):
    """遍歷所有 merged 資料夾內的檔案，去除極端值後輸出到 filtered 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            merged_path = os.path.join(recording_path, "out")  # 原始數據
            filtered_path = os.path.join(recording_path, "filtered")  # 存放處理後數據的資料夾

            if not os.path.exists(merged_path):
                continue  # 若 merged 資料夾不存在則跳過
            if not os.path.exists(filtered_path):
                os.makedirs(filtered_path)  # 建立 filtered 資料夾

            for file in sorted(os.listdir(merged_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(merged_path, file)
                    output_file = os.path.join(filtered_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_filtered = remove_outliers(df)  # 移除極端值

                        df_filtered.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Filtered: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

# Example usage
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'
# process_filtered_files(root_path)



In [65]:
##變化量
import os
import numpy as np
import pandas as pd

def compute_differences(df):
    """計算 DataFrame 每行與前一行的變化量，第一行補 0。"""
    delta_df = df.diff().fillna(0)  # 第一行 NaN 填補為 0
    return delta_df

def process_delta_files(root_path):
    """遍歷所有 out 資料夾內的 merged_ 檔案，計算變化量並輸出到 delta 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            out_path = os.path.join(recording_path, "filtered")
            delta_path = os.path.join(recording_path, "filtered_delta")

            if not os.path.exists(out_path):
                continue  # 若 out 資料夾不存在則跳過
            if not os.path.exists(delta_path):
                os.makedirs(delta_path)  # 建立 delta 資料夾

            for file in sorted(os.listdir(out_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(out_path, file)
                    output_file = os.path.join(delta_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_diff = compute_differences(df)  # 計算變化量

                        df_diff.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Δ: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

# Example usage
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'
# process_delta_files(root_path)


In [67]:
##二街變化量
import os
import pandas as pd

def compute_second_differences(df):
    """計算 DataFrame 每行與前一行的變化量的變化量，第一行補 0。"""
    delta2_df = df.diff().fillna(0)  # 計算變化量的變化量
    return delta2_df

def process_second_differences(root_path):
    """遍歷所有 delta 資料夾內的檔案，計算變化量的變化量並輸出到 delta2 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            delta_path = os.path.join(recording_path, "filtered_delta")
            delta2_path = os.path.join(recording_path, "filtered_delta_square")

            if not os.path.exists(delta_path):
                continue  # 若 delta 資料夾不存在則跳過
            if not os.path.exists(delta2_path):
                os.makedirs(delta2_path)  # 建立 delta2 資料夾

            for file in sorted(os.listdir(delta_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(delta_path, file)
                    output_file = os.path.join(delta2_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_delta2 = compute_second_differences(df)  # 計算變化量的變化量

                        df_delta2.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Δ²: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

# Example usage
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'
# process_second_differences(root_path)


In [69]:
##VRC
import os
import numpy as np
import pandas as pd

def compute_delta_ratio(data):
    """計算變化量與原始值的比 (B - A) / A"""
    epsilon = 1e-6  # 避免 A 為 0
    delta_ratio = (data[1:] - data[:-1]) / (data[:-1] + epsilon)
    delta_ratio = np.vstack([np.zeros((1, data.shape[1])) , delta_ratio])  # 第一行補 0
    return delta_ratio

def process_delta_ratio(root_path, input_folder="filtered", output_folder="filtered_delta2"):
    """讀取 `out/`，計算 (B - A) / A 並存入 `out_delta2/`"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue

            input_path = os.path.join(recording_path, input_folder)
            output_path = os.path.join(recording_path, output_folder)

            if not os.path.exists(input_path):
                continue  # 沒有這個資料夾就跳過
            
            os.makedirs(output_path, exist_ok=True)  # 確保輸出資料夾存在

            for file in sorted(os.listdir(input_path)):
                if file.startswith("merged_"):  # 只處理 merged_*.txt
                    file_path = os.path.join(input_path, file)
                    try:
                        df = pd.read_csv(file_path, header=None)
                        delta_ratio_data = compute_delta_ratio(df.to_numpy(dtype=np.float32))
                        pd.DataFrame(delta_ratio_data).to_csv(os.path.join(output_path, file), 
                                                              index=False, header=False, float_format="%.6f")
                        print(f"Processed delta ratio: {file_path} -> {output_path}/{file}")

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

# 設定根目錄
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'

# 計算 delta2
# process_delta_ratio(root_path)


In [71]:
##z score
import os
import pandas as pd

def z_score_normalization(df):
    """對 DataFrame 進行 Z-score 標準化，每一列 (特徵) 依據自身均值與標準差計算。"""
    mean = df.mean()
    std = df.std()
    
    std.replace(0, 1, inplace=True)  # 避免標準差為 0 導致除以 0
    zscore_df = (df - mean) / std
    return zscore_df

def process_zscore_from_merged(root_path):
    """遍歷所有 merged 資料夾內的檔案，對數據進行 Z-score 標準化並輸出到 zscore 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            merged_path = os.path.join(recording_path, "filtered")  # 原始數據
            zscore_path = os.path.join(recording_path, "filtered_zscore")  # 存放 Z-score 的資料夾

            if not os.path.exists(merged_path):
                continue  # 若 merged 資料夾不存在則跳過
            if not os.path.exists(zscore_path):
                os.makedirs(zscore_path)  # 建立 zscore 資料夾

            for file in sorted(os.listdir(merged_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(merged_path, file)
                    output_file = os.path.join(zscore_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_zscore = z_score_normalization(df)  # 計算 Z-score 標準化

                        df_zscore.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Z-score: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

# Example usage
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'
# process_zscore_from_merged(root_path)


In [10]:
##畫圖
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def read_category_data(root_path, category, target_length=110):
    """讀取該類別下所有 merged_*.txt，合併為統一格式，計算平均值與標準差"""
    category_path = os.path.join(root_path, category)
    all_data = []

    if not os.path.isdir(category_path):
        return None, None

    for recording in os.listdir(category_path):
        recording_path = os.path.join(category_path, recording, "filtered_delta2_norm")
        if not os.path.exists(recording_path):
            continue
        
        for file in sorted(os.listdir(recording_path)):
            if file.startswith("merged_"):  # 不管後面是什麼編號，統一處理
                file_path = os.path.join(recording_path, file)
                
                try:
                    df = pd.read_csv(file_path, header=None)
                    
                    # 確保行數正確
                    if df.shape[0] != target_length:
                        print(f"Skipping {file_path}: Unexpected row count {df.shape[0]}")
                        continue
                    
                    # 轉換為數值陣列
                    data_array = df.to_numpy(dtype=np.float32)

                    if len(all_data) > 0 and data_array.shape[1] != all_data[0].shape[1]:
                        print(f"Skipping {file_path}: Column mismatch {data_array.shape[1]} != {all_data[0].shape[1]}")
                        continue

                    all_data.append(data_array)
                
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")

    if len(all_data) == 0:
        return None, None

    all_data = np.stack(all_data, axis=0)  # 確保所有數據形狀一致
    return np.mean(all_data, axis=0), np.std(all_data, axis=0)  # 計算平均值與標準差

def plot_feature_comparison(root_path, save_path):
    """繪製 Category_1 和其他類別的數據對比，包含標準差區間"""
    categories = ["Category_1", "Category_2", "Category_3", "Category_4", "Category_5"]
    category_stats = {}

    # 讀取所有類別的數據 (平均值 + 標準差)
    for category in categories:
        print(f"Processing {category} ...")
        mean, std = read_category_data(root_path, category)
        category_stats[category] = (mean, std)

    # 確保 Category_1 存在
    if category_stats["Category_1"][0] is None:
        print("Category_1 資料缺失，無法比較！")
        return

    num_features = category_stats["Category_1"][0].shape[1]  # 取得特徵數量
    comparisons = ["Category_2", "Category_3", "Category_4", "Category_5"]

    # 創建存放圖片的資料夾
    os.makedirs(save_path, exist_ok=True)

    for feature_idx in range(num_features):  # 每一列 (特徵)
        for other_category in comparisons:  # 和其他類別比較
            if category_stats[other_category][0] is None:
                print(f"跳過 {other_category}，無數據！")
                continue

            # 取得平均值與標準差
            mean_1, std_1 = category_stats["Category_1"]
            mean_2, std_2 = category_stats[other_category]

            plt.figure(figsize=(8, 5))

            # 畫出 Category_1
            plt.plot(mean_1[:, feature_idx], label="Category_1", color='blue')
            plt.fill_between(range(mean_1.shape[0]), 
                             mean_1[:, feature_idx] - std_1[:, feature_idx], 
                             mean_1[:, feature_idx] + std_1[:, feature_idx], 
                             color='blue', alpha=0.2)

            # 畫出 Other Category
            plt.plot(mean_2[:, feature_idx], label=other_category, color='red')
            plt.fill_between(range(mean_2.shape[0]), 
                             mean_2[:, feature_idx] - std_2[:, feature_idx], 
                             mean_2[:, feature_idx] + std_2[:, feature_idx], 
                             color='red', alpha=0.2)

            plt.xlabel("Frame")
            plt.ylabel(f"Feature {feature_idx + 1}")
            plt.title(f"Feature {feature_idx + 1}: Category_1 vs {other_category}")
            plt.legend()
            plt.grid(True)

            # 存圖
            filename = f"feature_{feature_idx + 1}_Category_1_vs_{other_category}.png"
            plt.savefig(os.path.join(save_path, filename))
            plt.close()

            print(f"Saved: {filename}")

# Example usage
root_path = 'D:\\labdata\\MOCAP\\train_data'
save_path = 'D:\\labdata\\MOCAP\\train_data\\plots_filtered_delta2_norm'
plot_feature_comparison(root_path, save_path)


Processing Category_1 ...
Processing Category_2 ...
Processing Category_3 ...
Processing Category_4 ...
Processing Category_5 ...
Saved: feature_1_Category_1_vs_Category_2.png
Saved: feature_1_Category_1_vs_Category_3.png
Saved: feature_1_Category_1_vs_Category_4.png
Saved: feature_1_Category_1_vs_Category_5.png
Saved: feature_2_Category_1_vs_Category_2.png
Saved: feature_2_Category_1_vs_Category_3.png
Saved: feature_2_Category_1_vs_Category_4.png
Saved: feature_2_Category_1_vs_Category_5.png
Saved: feature_3_Category_1_vs_Category_2.png
Saved: feature_3_Category_1_vs_Category_3.png
Saved: feature_3_Category_1_vs_Category_4.png
Saved: feature_3_Category_1_vs_Category_5.png
Saved: feature_4_Category_1_vs_Category_2.png
Saved: feature_4_Category_1_vs_Category_3.png
Saved: feature_4_Category_1_vs_Category_4.png
Saved: feature_4_Category_1_vs_Category_5.png
Saved: feature_5_Category_1_vs_Category_2.png
Saved: feature_5_Category_1_vs_Category_3.png
Saved: feature_5_Category_1_vs_Category_4.

In [73]:
##正規化 -1~1
import os
import numpy as np
import pandas as pd
import math

def normalize_data(data):
    """將數據正規化到 [-1, 1]"""
    min_val = np.min(data, axis=0)
    max_val = np.max(data, axis=0)
    
    # 防止除以零
    range_val = max_val - min_val
    range_val[range_val == 0] = 1  # 避免 NaN，若 min==max，則保持為 0

    norm_data = 2 * (data - min_val) / range_val - 1
    return norm_data

def process_normalization(root_path, input_folder, output_folder):
    """讀取 out_delta/ 或 out_delta2/，正規化後存入新資料夾"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue

            input_path = os.path.join(recording_path, input_folder)
            output_path = os.path.join(recording_path, output_folder)

            if not os.path.exists(input_path):
                continue  # 沒有這個資料夾就跳過
            
            os.makedirs(output_path, exist_ok=True)  # 確保輸出資料夾存在

            for file in sorted(os.listdir(input_path)):
                if file.startswith("merged_"):  # 處理所有 merged_*.txt
                    file_path = os.path.join(input_path, file)
                    try:
                        df = pd.read_csv(file_path, header=None)
                        norm_data = normalize_data(df.to_numpy(dtype=np.float32))
                        pd.DataFrame(norm_data).to_csv(os.path.join(output_path, file), 
                                                       index=False, header=False, float_format="%.6f")
                        print(f"Normalized: {file_path} -> {output_path}/{file}")

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

# 設定根目錄
root_path = 'D:\\labdata\\MOCAP\\deadlift_system\\datasets'

# # # 處理 delta 和 delta2
# process_normalization(root_path, "filtered", "filtered_norm")
# process_normalization(root_path, "filtered_delta", "filtered_delta_norm")
# process_normalization(root_path, "filtered_delta2", "filtered_delta2_norm")
# process_normalization(root_path, "filtered_zscore", "filtered_zscore_norm")
# process_normalization(root_path, "filtered_delta_square", "filtered_delta_square_norm")



In [47]:
#放入ui
import tkinter as tk
from tkinter import filedialog
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import os
from scipy.signal import find_peaks, savgol_filter
import cv2
import math
import os
import numpy as np
import pandas as pd



def read_skeleton_data(filename):
    data = {}
    with open(filename, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 4:
                continue
            try:
                frame, joint, x, y = map(int, values[:4])
                if frame not in data:
                    data[frame] = {}
                data[frame][joint] = (x, y)
            except ValueError:
                continue
    return data

def calculate_angles(data):
    frames = sorted(data.keys())
    angles = []

    for frame in frames:
        joints = data[frame]
        if all(k in joints for k in [12, 14, 16]):
            angles.append(calculate_angle(joints[12], joints[14], joints[16]))
        else:
            angles.append(None)

    return frames, angles

def calculate_angle(p1, p2, p3):
    a, b, c = np.array(p1), np.array(p2), np.array(p3)
    ba, bc = a - b, c - b
    norm_ba = np.linalg.norm(ba)
    norm_bc = np.linalg.norm(bc)
    if norm_ba == 0 or norm_bc == 0:
        return 0
    cosine_angle = np.dot(ba, bc) / (norm_ba * norm_bc)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    return np.degrees(np.arccos(cosine_angle))

def find_valleys(smoothed_angles, peaks, search_range=10, min_valley_value=170, min_depth=10):
    valleys = []
    valleys1 = []

    for peak in peaks:
        # 在峰值前後各 `search_range` 幀內找最小值
        left_bound = max(0, peak - search_range)
        right_bound = min(len(smoothed_angles) - 1, peak + search_range)

        # 找到左邊和右邊的波谷
        left_min_index = left_bound + np.argmin(smoothed_angles[left_bound:peak])
        right_min_index = peak + np.argmin(smoothed_angles[peak:right_bound + 1])

        left_min_value = smoothed_angles[left_min_index]
        right_min_value = smoothed_angles[right_min_index]
        peak_value = smoothed_angles[peak]

        # 檢查條件：波谷值小於 170，且谷底夠深 (峰值 - 谷底 >= min_depth)
        if left_min_value < min_valley_value and (peak_value - left_min_value) >= min_depth:
            valleys.append(left_min_index)

        if right_min_value < min_valley_value and (peak_value - right_min_value) >= min_depth:
            valleys1.append(right_min_index)

    return valleys, valleys1

def read_yolo_data(yolo_file):
    """ 讀取 yolo_coordinates_interpolated.txt，回傳 {frame: x} 字典 """
    yolo_data = {}
    with open(yolo_file, 'r') as file:
        for line in file:
            values = line.strip().split(',')
            if len(values) < 3:
                continue
            try:
                frame = int(values[0])
                x_value = float(values[2])  # 第三個數值是 X 座標
                yolo_data[frame] = x_value
            except ValueError:
                continue
    return yolo_data

def split_video(video_path, output_folder, valleys, valleys1, yolo_file, fps=30):
    """
    根據 valleys (黃色谷底) 和 valleys1 (綠色谷底) 分割影片，
    **只存 X 座標變化超過 20 的影片**，如果變化 <= 20，則不存成影片。
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 讀取影片
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 格式

    # 讀取 yolo 數據
    yolo_data = read_yolo_data(yolo_file)

    for i, (start_frame, end_frame) in enumerate(zip(valleys, valleys1)):
        if start_frame >= end_frame or start_frame >= total_frames or end_frame >= total_frames:
            print(f"跳過無效範圍: 開始 {start_frame}, 結束 {end_frame}")
            continue

        # 取得該範圍內的 yolo X 值變化
        x_values = [yolo_data[f] for f in range(start_frame, end_frame + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0

        # **如果變化小於等於 20，則跳過該片段**
        if x_range <= 50 or np.isnan(x_range):
            print(f"跳過片段 {i+1}（X 變化範圍: {x_range:.2f}，過小）")
            continue

        # 存影片
        output_path = os.path.join(output_folder, f"clip_{i+1}.mp4")
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        for frame_idx in range(start_frame, end_frame + 1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        print(f"影片儲存: {output_path}（X 變化範圍: {x_range:.2f}）")

    cap.release()
    print("所有片段處理完成！")


def split_skeleton_data(original_skeleton_path, output_folder, valleys, valleys1, yolo_file):
    """
    根據 valleys (黃色谷底) 和 valleys1 (綠色谷底) 分割骨架數據，
    **只存 X 座標變化超過 20 的片段**，如果變化 <= 20，則不存。
    """
    # 讀取骨架數據檔案
    with open(original_skeleton_path, 'r') as file:
        lines = file.readlines()

    # 讀取 yolo 數據
    yolo_data = read_yolo_data(yolo_file)

    for i, (start_frame, end_frame) in enumerate(zip(valleys, valleys1)):
        if start_frame >= end_frame:
            print(f"跳過無效範圍: 開始 {start_frame}, 結束 {end_frame}")
            continue

        # 計算該片段內的 X 座標變化範圍
        x_values = [yolo_data[f] for f in range(start_frame, end_frame + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0

        # **如果變化小於等於 20，則跳過該片段**
        if x_range <= 50 or np.isnan(x_range):
            print(f"跳過骨架數據 {i+1}（X 變化範圍: {x_range:.2f}，過小）")
            continue

        # 存骨架數據
        clip_folder = os.path.join(output_folder)
        os.makedirs(clip_folder, exist_ok=True)

        output_skeleton_path = os.path.join(clip_folder, f"skeleton_{i+1}.txt")
        with open(output_skeleton_path, 'w') as out_file:
            for line in lines:
                values = line.strip().split(',')
                if len(values) < 4:
                    continue
                try:
                    frame = int(values[0])  # 解析 frame 數
                    if start_frame <= frame <= end_frame:
                        out_file.write(line)  # 保留符合範圍的行
                except ValueError:
                    continue

        print(f"骨架數據儲存: {output_skeleton_path}（Y 變化範圍: {x_range:.2f}）")

    print("所有骨架數據處理完成！")


def split_bar_data(original_bar_path, output_folder, valleys, valleys1, yolo_file):
    """
    根據 valleys (黃色谷底) 和 valleys1 (綠色谷底) 分割槓鈴數據，
    **只存 X 座標變化超過 20 的片段**，如果變化 ≤ 20，則不存。
    """
    # 讀取槓鈴數據檔案
    with open(original_bar_path, 'r') as file:
        lines = file.readlines()

    # 讀取 yolo 數據
    yolo_data = read_yolo_data(yolo_file)

    for i, (start_frame, end_frame) in enumerate(zip(valleys, valleys1)):
        if start_frame >= end_frame:
            print(f"跳過無效範圍: 開始 {start_frame}, 結束 {end_frame}")
            continue

        # 計算該片段內的 X 座標變化範圍
        x_values = [yolo_data[f] for f in range(start_frame, end_frame + 1) if f in yolo_data]
        x_range = max(x_values) - min(x_values) if x_values else 0

        # **如果變化小於等於 20，則跳過該片段**
        if x_range <= 50 or np.isnan(x_range):
            print(f"跳過槓鈴數據 {i+1}（X 變化範圍: {x_range:.2f}，過小）")
            continue

        # 存槓鈴數據
        clip_folder = os.path.join(output_folder)
        os.makedirs(clip_folder, exist_ok=True)

        output_bar_path = os.path.join(clip_folder, f"bar_{i+1}.txt")
        with open(output_bar_path, 'w') as out_file:
            for line in lines:
                values = line.strip().split(',')
                if len(values) < 4:
                    continue
                try:
                    frame = int(values[0])  # 解析 frame 數
                    if start_frame <= frame <= end_frame:
                        out_file.write(line)  # 保留符合範圍的行
                except ValueError:
                    continue

        print(f"槓鈴數據儲存: {output_bar_path}（X 變化範圍: {x_range:.2f}）")

    print("所有槓鈴數據處理完成！")

        
def plot_metrics_in_tkinter():
    global root, canvas, valleys, valleys1
    root = tk.Tk()
    root.title("Left Knee Angle Analysis")
    root.geometry("1000x600")

    fig, axes = Figure(figsize=(10, 6), dpi=100), [None, None]
    axes[0] = fig.add_subplot(2, 1, 1)  # 原始數據圖
    axes[1] = fig.add_subplot(2, 1, 2)  # 平滑後數據圖

    # 過濾掉 None 值
    valid_frames = [frames[i] for i in range(len(left_knee_angles)) if left_knee_angles[i] is not None]
    valid_angles = np.array([angle for angle in left_knee_angles if angle is not None])

    # **平滑化曲線**
    smoothed_angles = savgol_filter(valid_angles, window_length=11, polyorder=3)

    # 找出 170 度以上的峰值，間隔 10 幀以上
    peaks, _ = find_peaks(smoothed_angles, height=160, distance=55, prominence=5)

    # 找出峰值前後的谷底
    all_valleys, all_valleys1 = find_valleys(smoothed_angles, peaks, search_range=90, min_valley_value=170, min_depth=10)

    # 新的波谷列表
    valleys, valleys1 = [], []

    # 確保每個峰值左右都有波谷
    for peak in peaks:
        # 檢查左側的波谷
        left_valley = None
        for valley in all_valleys:
            if valley < peak:
                left_valley = valley
            else:
                break
        
        # 檢查右側的波谷
        right_valley = None
        for valley1 in all_valleys1:
            if valley1 > peak:
                right_valley = valley1
                break

        # 如果找到的左側和右側波谷都存在，則將其添加到各自的列表中
        if left_valley is not None and right_valley is not None:
            valleys.append(left_valley)
            valleys1.append(right_valley)

#     ======= 繪製原始數據 =======
#     axes[0].plot(valid_frames, valid_angles, color='gray', alpha=0.7)
#     axes[0].plot(np.array(valid_frames)[peaks], valid_angles[peaks], 'ro')  # 紅色標記峰值
#     axes[0].plot(np.array(valid_frames)[valleys], valid_angles[valleys], 'yo')  # 黃色標記谷底
#     axes[0].plot(np.array(valid_frames)[valleys1], valid_angles[valleys1], 'go')  # 綠色標記谷底
#     axes[0].set_title("Raw Left Knee Angle with Peaks and Valleys")
#     axes[0].set_xlabel("Frame")
#     axes[0].set_ylabel("Angle (degrees)")
#     axes[0].legend()
#     axes[0].grid()

#     # ======= 繪製平滑後數據 =======
#     axes[1].plot(valid_frames, smoothed_angles, color='blue')
#     axes[1].plot(np.array(valid_frames)[peaks], smoothed_angles[peaks], 'ro')
#     axes[1].plot(np.array(valid_frames)[valleys], smoothed_angles[valleys], 'yo')
#     axes[1].plot(np.array(valid_frames)[valleys1], smoothed_angles[valleys1], 'go')
#     axes[1].set_title("Smoothed Left Knee Angle with Peaks and Valleys")
#     axes[1].set_xlabel("Frame")
#     axes[1].set_ylabel("Angle (degrees)")
#     axes[1].legend()
#     axes[1].grid()

    print(valleys, valleys1)

#     canvas = FigureCanvasTkAgg(fig, root)
#     canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
#     root.mainloop()

def calculate_distance(x1, y1, x2, y2):
    """計算兩點之間的歐氏距離"""
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_angle1(x1, y1, x2, y2, x3, y3):
    """計算由三點構成的夾角，x2, y2 是頂點"""
    c = calculate_distance(x2, y2, x3, y3)
    a = calculate_distance(x1, y1, x3, y3)
    b = calculate_distance(x1, y1, x2, y2)

    if b == 0 or c == 0:
        return 0.0

    try:
        cos_theta = (b**2 + c**2 - a**2) / (2 * b * c)
        cos_theta = max(-1.0, min(1.0, cos_theta))
        angle = math.acos(cos_theta)
        return math.degrees(angle)
    except ValueError:
        return 0.0

def process_file(input_file, output_file):
    joint_data = {}

    try:
        # 讀取檔案並整理資料
        with open(input_file, 'r') as f:
            for line in f:
                frame, index, x, y = map(float, line.strip().split(','))
                frame = int(frame)
                index = int(index)
                if frame not in joint_data:
                    joint_data[frame] = {}
                joint_data[frame][index] = (x, y)

        # 計算角度和長度，並保存結果
        with open(output_file, 'w') as f_out:
#             f_out.write("frame,hip_angle,knee_angle,body_length,knee_hip_ratio\n")
            for frame, joints in sorted(joint_data.items()):
                if all(idx in joints for idx in [6, 12, 14, 16]):
                    x6, y6 = joints[6]
                    x12, y12 = joints[12]
                    x14, y14 = joints[14]
                    x16, y16 = joints[16]

                    hip_angle = calculate_angle1(x6, y6, x12, y12, x14, y14)
                    knee_angle = calculate_angle1(x12, y12, x14, y14, x16, y16)
                    body_length = calculate_distance(x6, y6, x12, y12)
                    knee_hip_ratio = knee_angle / hip_angle if hip_angle != 0 else 0

                    f_out.write(f"{frame},{hip_angle:.2f},{knee_angle:.2f},{body_length:.2f},{knee_hip_ratio:.2f}\n")
    except Exception as e:
        print(f"處理檔案時發生錯誤：{input_file}，錯誤訊息：{e}")

def process_all_folders(base_path):
    # 遞迴遍歷所有子資料夾
    for root, dirs, files in os.walk(base_path):
        for filename in files:
            if filename.startswith("skeleton_") and filename.endswith(".txt"):
                input_file = os.path.join(root, filename)
                
                parent_folder = os.path.dirname(root)
                output_folder = os.path.join(parent_folder, "angle")
                os.makedirs(output_folder, exist_ok=True)

                # 輸出檔案路徑
                output_file = os.path.join(output_folder, f"angle_{filename}")
                process_file(input_file, output_file)
                print(f"已處理檔案：{input_file}，結果保存至：{output_file}")

def linear_interpolate_fixed_length(df, frame_col=0, target_length=110):
    """對 DataFrame 進行線性內插，使其長度固定為 target_length。"""
    df.set_index(df.columns[frame_col], inplace=True)  # 設定時間軸為索引
    new_index = np.linspace(df.index.min(), df.index.max(), target_length)
    df = df.reindex(df.index.union(new_index)).interpolate(method='linear').loc[new_index]
    df.reset_index(drop=True, inplace=True)
    return df

def merge_and_interpolate(angle_path, bar_path, output_path, target_length=110):
    """合併 angle 和 bar 資料夾內的檔案，並進行線性內插。"""
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    angle_files = sorted([f for f in os.listdir(angle_path) if f.startswith("angle_skeleton_")])
    bar_files = sorted([f for f in os.listdir(bar_path) if f.startswith("bar_")])
    
    # 確保檔案名稱匹配
    angle_dict = {f.split("_")[-1]: f for f in angle_files}
    bar_dict = {f.split("_")[-1]: f for f in bar_files}
    
    common_keys = set(angle_dict.keys()) & set(bar_dict.keys())  # 找到對應的檔案

    for key in sorted(common_keys):
        angle_file = angle_dict[key]
        bar_file = bar_dict[key]

        try:
            angle_df = pd.read_csv(os.path.join(angle_path, angle_file), header=None)
            bar_df = pd.read_csv(os.path.join(bar_path, bar_file), header=None)
            
            # 選擇需要的欄位
            angle_df = angle_df.iloc[:, [0, 1, 2, 3]]  # 包含 frame_col (假設是第 0 欄)
            bar_df = bar_df.iloc[:, [1, 2]]

            merged_df = pd.concat([angle_df, bar_df], axis=1)

            # 進行內插，確保 frame_col 為索引
            merged_df = linear_interpolate_fixed_length(merged_df, frame_col=0, target_length=target_length)

            output_file = os.path.join(output_path, f"merged_{key}")
            merged_df.to_csv(output_file, index=False, header=False, float_format="%.2f")

            print(f"Processed: {angle_file} + {bar_file} -> {output_file}")

        except Exception as e:
            print(f"Error processing {angle_file} and {bar_file}: {e}")
def process_all_folders1(root_path, target_length=110):
    """遍歷 root_path 內的所有子資料夾，並處理其中的 angle 和 bar。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            angle_path = os.path.join(recording_path, "angle")
            bar_path = os.path.join(recording_path, "bar")
            output_path = os.path.join(recording_path, "out")

            if os.path.exists(angle_path) and os.path.exists(bar_path):
                print(f"Processing {recording_path} ...")
                merge_and_interpolate(angle_path, bar_path, output_path, target_length)

def remove_outliers(df):
    """移除超過3個標準差的極端值，並用該列均值取代。"""
    mean = df.mean()
    std = df.std()
    
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std

    # 過濾掉極端值，超過範圍的值用均值取代
    df_filtered = df.mask((df < lower_bound) | (df > upper_bound), mean, axis=1)
    return df_filtered

def process_filtered_files(root_path):
    """遍歷所有 merged 資料夾內的檔案，去除極端值後輸出到 filtered 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            merged_path = os.path.join(recording_path, "out")  # 原始數據
            filtered_path = os.path.join(recording_path, "data/filtered")  # 存放處理後數據的資料夾

            if not os.path.exists(merged_path):
                continue  # 若 merged 資料夾不存在則跳過
            if not os.path.exists(filtered_path):
                os.makedirs(filtered_path)  # 建立 filtered 資料夾

            for file in sorted(os.listdir(merged_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(merged_path, file)
                    output_file = os.path.join(filtered_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_filtered = remove_outliers(df)  # 移除極端值

                        df_filtered.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Filtered: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

def compute_differences(df):
    """計算 DataFrame 每行與前一行的變化量，第一行補 0。"""
    delta_df = df.diff().fillna(0)  # 第一行 NaN 填補為 0
    return delta_df

def process_delta_files(root_path):
    """遍歷所有 out 資料夾內的 merged_ 檔案，計算變化量並輸出到 delta 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            out_path = os.path.join(recording_path, "data\\filtered")
            delta_path = os.path.join(recording_path, "data\\filtered_delta")

            if not os.path.exists(out_path):
                continue  # 若 out 資料夾不存在則跳過
            if not os.path.exists(delta_path):
                os.makedirs(delta_path)  # 建立 delta 資料夾

            for file in sorted(os.listdir(out_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(out_path, file)
                    output_file = os.path.join(delta_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_diff = compute_differences(df)  # 計算變化量

                        df_diff.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Δ: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

def compute_second_differences(df):
    """計算 DataFrame 每行與前一行的變化量的變化量，第一行補 0。"""
    delta2_df = df.diff().fillna(0)  # 計算變化量的變化量
    return delta2_df

def process_second_differences(root_path):
    """遍歷所有 delta 資料夾內的檔案，計算變化量的變化量並輸出到 delta2 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            delta_path = os.path.join(recording_path, "data/filtered_delta")
            delta2_path = os.path.join(recording_path, "data/filtered_delta_square")

            if not os.path.exists(delta_path):
                continue  # 若 delta 資料夾不存在則跳過
            if not os.path.exists(delta2_path):
                os.makedirs(delta2_path)  # 建立 delta2 資料夾

            for file in sorted(os.listdir(delta_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(delta_path, file)
                    output_file = os.path.join(delta2_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_delta2 = compute_second_differences(df)  # 計算變化量的變化量

                        df_delta2.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Δ²: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

def compute_delta_ratio(data):
    """計算變化量與原始值的比 (B - A) / A"""
    epsilon = 1e-6  # 避免 A 為 0
    delta_ratio = (data[1:] - data[:-1]) / (data[:-1] + epsilon)
    delta_ratio = np.vstack([np.zeros((1, data.shape[1])) , delta_ratio])  # 第一行補 0
    return delta_ratio

def process_delta_ratio(root_path, input_folder="data\\filtered", output_folder="data\\filtered_delta2"):
    """讀取 `out/`，計算 (B - A) / A 並存入 `out_delta2/`"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue

            input_path = os.path.join(recording_path, input_folder)
            output_path = os.path.join(recording_path, output_folder)

            if not os.path.exists(input_path):
                continue  # 沒有這個資料夾就跳過
            
            os.makedirs(output_path, exist_ok=True)  # 確保輸出資料夾存在

            for file in sorted(os.listdir(input_path)):
                if file.startswith("merged_"):  # 只處理 merged_*.txt
                    file_path = os.path.join(input_path, file)
                    try:
                        df = pd.read_csv(file_path, header=None)
                        delta_ratio_data = compute_delta_ratio(df.to_numpy(dtype=np.float32))
                        pd.DataFrame(delta_ratio_data).to_csv(os.path.join(output_path, file), 
                                                              index=False, header=False, float_format="%.6f")
                        print(f"Processed delta ratio: {file_path} -> {output_path}/{file}")

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

def z_score_normalization(df):
    """對 DataFrame 進行 Z-score 標準化，每一列 (特徵) 依據自身均值與標準差計算。"""
    mean = df.mean()
    std = df.std()
    
    std.replace(0, 1, inplace=True)  # 避免標準差為 0 導致除以 0
    zscore_df = (df - mean) / std
    return zscore_df

def process_zscore_from_merged(root_path):
    """遍歷所有 merged 資料夾內的檔案，對數據進行 Z-score 標準化並輸出到 zscore 資料夾。"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾的檔案

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue
            
            merged_path = os.path.join(recording_path, "data\\filtered")  # 原始數據
            zscore_path = os.path.join(recording_path, "data\\filtered_zscore")  # 存放 Z-score 的資料夾

            if not os.path.exists(merged_path):
                continue  # 若 merged 資料夾不存在則跳過
            if not os.path.exists(zscore_path):
                os.makedirs(zscore_path)  # 建立 zscore 資料夾

            for file in sorted(os.listdir(merged_path)):
                if file.startswith("merged_"):  # 只處理 merged_ 開頭的檔案
                    input_file = os.path.join(merged_path, file)
                    output_file = os.path.join(zscore_path, file)

                    try:
                        df = pd.read_csv(input_file, header=None)
                        df_zscore = z_score_normalization(df)  # 計算 Z-score 標準化

                        df_zscore.to_csv(output_file, index=False, header=False, float_format="%.4f")
                        print(f"Processed Z-score: {input_file} -> {output_file}")

                    except Exception as e:
                        print(f"Error processing {input_file}: {e}")

def normalize_data(data):
    """將數據正規化到 [-1, 1]"""
    min_val = np.min(data, axis=0)
    max_val = np.max(data, axis=0)
    
    # 防止除以零
    range_val = max_val - min_val
    range_val[range_val == 0] = 1  # 避免 NaN，若 min==max，則保持為 0

    norm_data = 2 * (data - min_val) / range_val - 1
    return norm_data

def process_normalization(root_path, input_folder, output_folder):
    """讀取 out_delta/ 或 out_delta2/，正規化後存入新資料夾"""
    for category in os.listdir(root_path):
        category_path = os.path.join(root_path, category)
        if not os.path.isdir(category_path):
            continue  # 跳過非資料夾

        for recording in os.listdir(category_path):
            recording_path = os.path.join(category_path, recording)
            if not os.path.isdir(recording_path):
                continue

            input_path = os.path.join(recording_path, input_folder)
            output_path = os.path.join(recording_path, output_folder)

            if not os.path.exists(input_path):
                continue  # 沒有這個資料夾就跳過
            
            os.makedirs(output_path, exist_ok=True)  # 確保輸出資料夾存在

            for file in sorted(os.listdir(input_path)):
                if file.startswith("merged_"):  # 處理所有 merged_*.txt
                    file_path = os.path.join(input_path, file)
                    try:
                        df = pd.read_csv(file_path, header=None)
                        norm_data = normalize_data(df.to_numpy(dtype=np.float32))
                        pd.DataFrame(norm_data).to_csv(os.path.join(output_path, file), 
                                                       index=False, header=False, float_format="%.6f")
                        print(f"Normalized: {file_path} -> {output_path}/{file}")

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

# 指定最上層資料夾
base_path = "D:/labdata/MOCAP/test/Category_1/"
root_path = 'D:\\labdata\\MOCAP\\test\\'
# 取得 base_path 下的所有子資料夾
for sub_folder in os.listdir(base_path):
    path = os.path.join(base_path, sub_folder)

    # 確保 path 是資料夾
    if not os.path.isdir(path):
        continue


    # 設定檔案路徑
    skeleton_file_path = os.path.join(path, 'yolo_skeleton_coordinates_1st_interp.txt')
    bar_file_path = os.path.join(path, 'yolo_coordinates_interpolated.txt')
    video_path = os.path.join(path, 'vision1.avi')
    output_folder = os.path.join(path, 'clips')
    output_folder1 = os.path.join(path, 'skeleton')

    output_folder2 = os.path.join(path, 'bar')

    # 檢查檔案是否存在，避免錯誤
    if not os.path.exists(skeleton_file_path):
        print(f"檔案 {skeleton_file_path} 不存在，跳過該資料夾")
        continue

#         讀取數據
    skeleton_data = read_skeleton_data(skeleton_file_path)
    frames, left_knee_angles = calculate_angles(skeleton_data)

    # 顯示波形圖
    plot_metrics_in_tkinter()
    split_video(video_path, output_folder, valleys, valleys1,bar_file_path, fps=30)
    # 執行切割
    split_skeleton_data(skeleton_file_path, output_folder1, valleys, valleys1,bar_file_path)
    split_bar_data(bar_file_path, output_folder2, valleys, valleys1,bar_file_path)
    

process_all_folders(base_path)



# Example usage
process_all_folders1(root_path, target_length=110)
process_filtered_files(root_path)
process_delta_files(root_path)
process_second_differences(root_path)
process_delta_ratio(root_path)
process_zscore_from_merged(root_path)
# 處理 delta 和 delta2
process_normalization(root_path, "data\\filtered", "data1\\filtered_norm")
process_normalization(root_path, "data\\filtered_delta", "data1\\filtered_delta_norm")
process_normalization(root_path, "data\\filtered_delta2", "data1\\filtered_delta2_norm")
process_normalization(root_path, "data\\filtered_zscore", "data1\\filtered_zscore_norm")
process_normalization(root_path, "data\\filtered_delta_square", "data1\\filtered_delta_square_norm")


[27, 125, 224, 315, 399, 489, 585, 675, 763, 851, 933, 1018, 1088, 1175, 1257, 1343, 1420] [125, 224, 315, 399, 489, 585, 675, 763, 851, 933, 1018, 1088, 1175, 1257, 1343, 1420, 1497]
跳過片段 1（X 變化範圍: 0.36，過小）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_2.mp4（X 變化範圍: 364.88）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_3.mp4（X 變化範圍: 326.36）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_4.mp4（X 變化範圍: 333.93）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_5.mp4（X 變化範圍: 320.22）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_6.mp4（X 變化範圍: 319.51）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_7.mp4（X 變化範圍: 308.37）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_8.mp4（X 變化範圍: 307.17）
影片儲存: D:/labdata/MOCAP/test/Category_1/recording_20241204_100527\clips\clip_9.mp4（X 變化範圍: 316.39）
影片儲存: D:

In [74]:
##把要用的資料複製到令一個資料夾
import os
import shutil

# 原始根目錄與目標根目錄
source_root = r'D:\\labdata\\MOCAP\\deadlift_system\\datasets'
target_root = r'D:\\labdata\\MOCAP\\2D_traindata_with_hand'

# 目標要複製的資料夾名稱
target_dirs = {
    'filtered_delta2_norm',
    'filtered_norm',
    'filtered_delta_square_norm',
    'filtered_zscore_norm',
    'filtered_delta_norm'
}

for category in os.listdir(source_root):
    category_path = os.path.join(source_root, category)
    for recording in os.listdir(category_path):
        recording_path = os.path.join(category_path, recording)
        print(recording_path)
    
        if os.path.isdir(recording_path):
            for folder in os.listdir(recording_path):
                if folder in target_dirs:
                    src = os.path.join(recording_path, folder)
                    relative_path = os.path.relpath(src, source_root)
                    dst = os.path.join(target_root, relative_path)
    
                    os.makedirs(os.path.dirname(dst), exist_ok=True)
                    shutil.copytree(src, dst, dirs_exist_ok=True)
                    print(f"複製成功：{src} -> {dst}")



D:\\labdata\\MOCAP\\deadlift_system\\datasets\Category_1\recording_20241121_101001
複製成功：D:\\labdata\\MOCAP\\deadlift_system\\datasets\Category_1\recording_20241121_101001\filtered_delta2_norm -> D:\\labdata\\MOCAP\\2D_traindata_with_hand\Category_1\recording_20241121_101001\filtered_delta2_norm
複製成功：D:\\labdata\\MOCAP\\deadlift_system\\datasets\Category_1\recording_20241121_101001\filtered_delta_norm -> D:\\labdata\\MOCAP\\2D_traindata_with_hand\Category_1\recording_20241121_101001\filtered_delta_norm
複製成功：D:\\labdata\\MOCAP\\deadlift_system\\datasets\Category_1\recording_20241121_101001\filtered_delta_square_norm -> D:\\labdata\\MOCAP\\2D_traindata_with_hand\Category_1\recording_20241121_101001\filtered_delta_square_norm
複製成功：D:\\labdata\\MOCAP\\deadlift_system\\datasets\Category_1\recording_20241121_101001\filtered_norm -> D:\\labdata\\MOCAP\\2D_traindata_with_hand\Category_1\recording_20241121_101001\filtered_norm
複製成功：D:\\labdata\\MOCAP\\deadlift_system\\datasets\Category_1\recordi

NotADirectoryError: [WinError 267] 目錄名稱無效。: 'D:\\\\labdata\\\\MOCAP\\\\deadlift_system\\\\datasets\\datasets.zip'

In [77]:
##移動2D壞掉檔案
import os
import shutil

# 設定原始目錄替換和目標根目錄
old_prefix = r'C:\Users\Cup\naruto\DLmodel_train'
new_prefix = r'D:\labdata\MOCAP\2D_traindata_with_hand'
destination_root = r'D:\\labdata\\MOCAP\\collected_recordings'  # 放移動結果的資料夾

# 載入TXT檔路徑
txt_path = r'C:\\Users\\user\\Downloads\\missing_merge.txt'  # ←請改成你的TXT檔位置

# 記錄已處理的recording資料夾，避免重複
recording_dirs = set()

with open(txt_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line in lines:
    line = line.strip()
    if not line:
        continue

    # 確保是合法路徑
    if old_prefix not in line:
        continue

    # 換路徑前綴
    new_path = line.replace(old_prefix, new_prefix)
    
    # 找出recording路徑
    parts = new_path.split(os.sep)
    try:
        cat_index = parts.index(next(p for p in parts if p.startswith("Category_")))  # 你可以改成更一般化，如 Category_ 開頭都接受
    except ValueError:
        continue

    category_folder = parts[cat_index]
    recording_folder = parts[cat_index + 1]
    full_recording_path = os.path.join(new_prefix, category_folder, recording_folder)

    # 避免重複處理
    if full_recording_path in recording_dirs:
        continue

    # 檢查該recording資料夾存在
    if not os.path.isdir(full_recording_path):
        print(f"[!] 資料夾不存在: {full_recording_path}")
        continue

    # 準備目標資料夾
    target_path = os.path.join(destination_root, category_folder, recording_folder)
    os.makedirs(os.path.dirname(target_path), exist_ok=True)

    # 複製整個recording資料夾
    try:
        shutil.move(full_recording_path, target_path)
        print(f"[✓] 已複製: {full_recording_path} → {target_path}")
        recording_dirs.add(full_recording_path)
    except FileExistsError:
        print(f"[!] 已存在: {target_path}，跳過")
    except Exception as e:
        print(f"[X] 複製失敗: {full_recording_path}，錯誤：{e}")


[✓] 已複製: D:\labdata\MOCAP\2D_traindata_with_hand\Category_1\recording_20241125_163257 → D:\\labdata\\MOCAP\\collected_recordings\Category_1\recording_20241125_163257
[✓] 已複製: D:\labdata\MOCAP\2D_traindata_with_hand\Category_1\recording_20241127_103044 → D:\\labdata\\MOCAP\\collected_recordings\Category_1\recording_20241127_103044
[✓] 已複製: D:\labdata\MOCAP\2D_traindata_with_hand\Category_1\recording_20241203_131457 → D:\\labdata\\MOCAP\\collected_recordings\Category_1\recording_20241203_131457
[✓] 已複製: D:\labdata\MOCAP\2D_traindata_with_hand\Category_1\recording_20241205_141420 → D:\\labdata\\MOCAP\\collected_recordings\Category_1\recording_20241205_141420
[✓] 已複製: D:\labdata\MOCAP\2D_traindata_with_hand\Category_1\recording_20241206_113600 → D:\\labdata\\MOCAP\\collected_recordings\Category_1\recording_20241206_113600
[✓] 已複製: D:\labdata\MOCAP\2D_traindata_with_hand\Category_1\recording_20241209_111652 → D:\\labdata\\MOCAP\\collected_recordings\Category_1\recording_20241209_111652
[✓] 